In [1]:
#!pip install lightgbm xgboost catboost category-encoders sklearn pandas

In [2]:
#!git clone https://github.com/tolarteh/cardashians.git

根据第六章所讲述的调参方式，请使用 LightGBM 进行调参：

目标变量：loan_status

要求：

1. 请在word当中记录详细的调参过程，包括参数和结果；
2. 请解释选择该调参步骤的原因；
3. 请报告在测试集上的准确率；
4. （可选）尝试不同的模型的组合，检查是否可以提升。

作业提交截止时间：7月8日 23:59
作业提交链接：https://jinshuju.net/f/DvsHN1

In [1]:
#读取数据
import pandas as pd
import numpy as np

test_final = pd.read_csv('./test_final.csv', engine='python')
train_final = pd.read_csv('./train_final.csv', engine='python')

In [20]:
#查看变量属性
#print(test_final.columns.to_list())
#print(train_final.columns.to_list())
#print(train_final.info())
#显示所有列
pd.set_option('display.max_columns', None)
train_final.dtypes

continuous_annual_inc                     float64
continuous_annual_inc_joint               float64
continuous_delinq_2yrs                    float64
continuous_dti                            float64
continuous_dti_joint                      float64
continuous_fico_range_high                float64
continuous_fico_range_low                 float64
continuous_funded_amnt                    float64
continuous_funded_amnt_inv                float64
continuous_inq_last_6mths                 float64
continuous_installment                    float64
continuous_int_rate                       float64
continuous_last_fico_range_high           float64
continuous_last_fico_range_low            float64
continuous_loan_amnt                      float64
loan_status                                 int64
continuous_mths_since_last_delinq         float64
continuous_mths_since_last_major_derog    float64
continuous_mths_since_last_record         float64
continuous_open_acc                       float64


In [21]:
train_final.describe()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,discrete_addr_state_20_one_hot,discrete_addr_state_21_one_hot,discrete_addr_state_22_one_hot,discrete_addr_state_23_one_hot,discrete_addr_state_24_one_hot,discrete_addr_state_25_one_hot,discrete_addr_state_26_one_hot,discrete_addr_state_27_one_hot,discrete_addr_state_28_one_hot,discrete_addr_state_29_one_hot,discrete_addr_state_30_one_hot,discrete_addr_state_31_one_hot,discrete_addr_state_32_one_hot,discrete_addr_state_33_one_hot,discrete_addr_state_34_one_hot,discrete_addr_state_35_one_hot,discrete_addr_state_36_one_hot,discrete_addr_state_37_one_hot,discrete_addr_state_38_one_hot,discrete_addr_state_39_one_hot,discrete_addr_state_40_one_hot,discrete_addr_state_41_one_hot,discrete_addr_state_42_one_hot,discrete_addr_state_43_one_hot,discrete_addr_state_44_one_hot,discrete_addr_state_45_one_hot,discrete_addr_state_46_one_hot,discrete_addr_state_47_one_hot,discrete_addr_state_48_one_hot,discrete_addr_state_49_one_hot,discrete_application_type_1_one_hot,discrete_application_type_2_one_hot,discrete_emp_length_1_one_hot,discrete_emp_length_2_one_hot,discrete_emp_length_3_one_hot,discrete_emp_length_4_one_hot,discrete_emp_length_5_one_hot,discrete_emp_length_6_one_hot,discrete_emp_length_7_one_hot,discrete_emp_length_8_one_hot,discrete_emp_length_9_one_hot,discrete_emp_length_10_one_hot,discrete_emp_length_11_one_hot,discrete_emp_length_12_one_hot,discrete_grade_1_one_hot,discrete_grade_2_one_hot,discrete_grade_3_one_hot,discrete_grade_4_one_hot,discrete_grade_5_one_hot,discrete_grade_6_one_hot,discrete_grade_7_one_hot,discrete_home_ownership_1_one_hot,discrete_home_ownership_2_one_hot,discrete_home_ownership_3_one_hot,discrete_home_ownership_4_one_hot,discrete_policy_code_1_one_hot,discrete_purpose_1_one_hot,discrete_purpose_2_one_hot,discrete_purpose_3_one_hot,discrete_purpose_4_one_hot,discrete_purpose_5_one_hot,discrete_purpose_6_one_hot,discrete_purpose_7_one_hot,discrete_purpose_8_one_hot,discrete_purpose_9_one_hot,discrete_purpose_10_one_hot,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,d

In [17]:
#查看目标变量是否均衡:
#轻微不均衡4:1
#导入用于计数的包
from collections import Counter
print('训练集')
print(Counter(train_final['loan_status']))
print('验证集')
print(Counter(test_final['loan_status']))

训练集
Counter({1: 39788, 0: 10212})
验证集
Counter({1: 40226, 0: 9774})


## 一.建立基准的TreeBased Models  

---
In this example, we use lightgbm as the tree model of choice.

In [42]:
#将老师提供的训练集数据划分成训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_final.drop(columns='loan_status'),train_final['loan_status'], test_size=0.2,stratify=train_final['loan_status'])
train = pd.concat([X_train,y_train], axis=1)
test = pd.concat([X_test,y_test], axis=1)
print('训练集和测试集样本量')
print(len(train))
print(len(test))

训练集和测试集样本量
40000
10000


In [48]:
import lightgbm as lgb
train_dataset = lgb.Dataset(train.drop(columns='loan_status'), train['loan_status'])
test_dataset = lgb.Dataset(test.drop(columns='loan_status'), test['loan_status'])

In [49]:
param = {'num_leaves': 31, 'objective': 'binary', 'metric':'binary_error'}
num_round = 2000

In [50]:
model = lgb.train(param, train_dataset, num_boost_round=num_round, valid_sets=[train_dataset, test_dataset])

[LightGBM] [Info] Number of positive: 31830, number of negative: 8170
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006625 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2579
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 141
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.795750 -> initscore=1.359940
[LightGBM] [Info] Start training from score 1.359940
[1]	training's binary_error: 0.20425	valid_1's binary_error: 0.2042
[2]	training's binary_error: 0.20425	valid_1's binary_error: 0.2042
[3]	training's binary_error: 0.20425	valid_1's binary_error: 0.2042
[4]	training's binary_error: 0.20425	valid_1's binary_error: 0.2042
[5]	training's binary_error: 0.15065	valid_1's binary_error: 0.1461
[6]	training's binary_error: 0.108925	valid_1's binary_error: 0.1071
[7]	training's binary_error: 0.0

[120]	training's binary_error: 0.0615	valid_1's binary_error: 0.0787
[121]	training's binary_error: 0.06125	valid_1's binary_error: 0.0785
[122]	training's binary_error: 0.061125	valid_1's binary_error: 0.0785
[123]	training's binary_error: 0.0611	valid_1's binary_error: 0.0784
[124]	training's binary_error: 0.061	valid_1's binary_error: 0.0784
[125]	training's binary_error: 0.06095	valid_1's binary_error: 0.0785
[126]	training's binary_error: 0.06065	valid_1's binary_error: 0.0786
[127]	training's binary_error: 0.06065	valid_1's binary_error: 0.0786
[128]	training's binary_error: 0.060475	valid_1's binary_error: 0.0787
[129]	training's binary_error: 0.06045	valid_1's binary_error: 0.0787
[130]	training's binary_error: 0.060375	valid_1's binary_error: 0.0786
[131]	training's binary_error: 0.06025	valid_1's binary_error: 0.0786
[132]	training's binary_error: 0.060075	valid_1's binary_error: 0.0787
[133]	training's binary_error: 0.059925	valid_1's binary_error: 0.0788
[134]	training's bi

[259]	training's binary_error: 0.044475	valid_1's binary_error: 0.0805
[260]	training's binary_error: 0.044475	valid_1's binary_error: 0.0804
[261]	training's binary_error: 0.04435	valid_1's binary_error: 0.0806
[262]	training's binary_error: 0.0443	valid_1's binary_error: 0.0806
[263]	training's binary_error: 0.0441	valid_1's binary_error: 0.0808
[264]	training's binary_error: 0.044075	valid_1's binary_error: 0.0807
[265]	training's binary_error: 0.043925	valid_1's binary_error: 0.0805
[266]	training's binary_error: 0.0439	valid_1's binary_error: 0.0805
[267]	training's binary_error: 0.043725	valid_1's binary_error: 0.0803
[268]	training's binary_error: 0.043575	valid_1's binary_error: 0.0803
[269]	training's binary_error: 0.0435	valid_1's binary_error: 0.0802
[270]	training's binary_error: 0.04345	valid_1's binary_error: 0.0802
[271]	training's binary_error: 0.043375	valid_1's binary_error: 0.0803
[272]	training's binary_error: 0.043225	valid_1's binary_error: 0.0803
[273]	training's

[382]	training's binary_error: 0.0307	valid_1's binary_error: 0.0815
[383]	training's binary_error: 0.030575	valid_1's binary_error: 0.0815
[384]	training's binary_error: 0.03045	valid_1's binary_error: 0.0814
[385]	training's binary_error: 0.03035	valid_1's binary_error: 0.0811
[386]	training's binary_error: 0.03025	valid_1's binary_error: 0.0811
[387]	training's binary_error: 0.03005	valid_1's binary_error: 0.081
[388]	training's binary_error: 0.029975	valid_1's binary_error: 0.0811
[389]	training's binary_error: 0.0299	valid_1's binary_error: 0.0813
[390]	training's binary_error: 0.02985	valid_1's binary_error: 0.0813
[391]	training's binary_error: 0.029675	valid_1's binary_error: 0.0814
[392]	training's binary_error: 0.029525	valid_1's binary_error: 0.0813
[393]	training's binary_error: 0.029425	valid_1's binary_error: 0.0814
[394]	training's binary_error: 0.029275	valid_1's binary_error: 0.0815
[395]	training's binary_error: 0.0293	valid_1's binary_error: 0.0816
[396]	training's b

[504]	training's binary_error: 0.020425	valid_1's binary_error: 0.0826
[505]	training's binary_error: 0.02035	valid_1's binary_error: 0.0826
[506]	training's binary_error: 0.02035	valid_1's binary_error: 0.0827
[507]	training's binary_error: 0.0201	valid_1's binary_error: 0.0827
[508]	training's binary_error: 0.0201	valid_1's binary_error: 0.0828
[509]	training's binary_error: 0.02005	valid_1's binary_error: 0.0825
[510]	training's binary_error: 0.01995	valid_1's binary_error: 0.0825
[511]	training's binary_error: 0.019875	valid_1's binary_error: 0.0823
[512]	training's binary_error: 0.0198	valid_1's binary_error: 0.0823
[513]	training's binary_error: 0.019725	valid_1's binary_error: 0.0822
[514]	training's binary_error: 0.0197	valid_1's binary_error: 0.0824
[515]	training's binary_error: 0.019625	valid_1's binary_error: 0.0824
[516]	training's binary_error: 0.019425	valid_1's binary_error: 0.0825
[517]	training's binary_error: 0.01935	valid_1's binary_error: 0.0825
[518]	training's bi

...

[1424]	training's binary_error: 0.0004	valid_1's binary_error: 0.0836
[1425]	training's binary_error: 0.0004	valid_1's binary_error: 0.0836
[1426]	training's binary_error: 0.0004	valid_1's binary_error: 0.0837
[1427]	training's binary_error: 0.0004	valid_1's binary_error: 0.0838
[1428]	training's binary_error: 0.000375	valid_1's binary_error: 0.0838
[1429]	training's binary_error: 0.000375	valid_1's binary_error: 0.0839
[1430]	training's binary_error: 0.000375	valid_1's binary_error: 0.0839
[1431]	training's binary_error: 0.000375	valid_1's binary_error: 0.0839
[1432]	training's binary_error: 0.000375	valid_1's binary_error: 0.0839
[1433]	training's binary_error: 0.000375	valid_1's binary_error: 0.0839
[1434]	training's binary_error: 0.000375	valid_1's binary_error: 0.0838
[1435]	training's binary_error: 0.000375	valid_1's binary_error: 0.0838
[1436]	training's binary_error: 0.000375	valid_1's binary_error: 0.0838
[1437]	training's binary_error: 0.000375	valid_1's binary_error: 0.0838


[1556]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1557]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1558]	training's binary_error: 0.000125	valid_1's binary_error: 0.0839
[1559]	training's binary_error: 0.000125	valid_1's binary_error: 0.0839
[1560]	training's binary_error: 0.000125	valid_1's binary_error: 0.0838
[1561]	training's binary_error: 0.000125	valid_1's binary_error: 0.0838
[1562]	training's binary_error: 0.000125	valid_1's binary_error: 0.0838
[1563]	training's binary_error: 0.000125	valid_1's binary_error: 0.0837
[1564]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1565]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1566]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1567]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1568]	training's binary_error: 0.000125	valid_1's binary_error: 0.0836
[1569]	training's binary_error: 0.000125	valid_1's binary_error:

[1686]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0834
[1687]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0835
[1688]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0835
[1689]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0835
[1690]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0836
[1691]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0836
[1692]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0835
[1693]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0836
[1694]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0836
[1695]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0837
[1696]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0837
[1697]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0837
[1698]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0836
[1699]	training's binary_error: 2.5e-05	valid_1's binary_error: 0.0834
[1700]

[1813]	training's binary_error: 5e-05	valid_1's binary_error: 0.0833
[1814]	training's binary_error: 5e-05	valid_1's binary_error: 0.0831
[1815]	training's binary_error: 5e-05	valid_1's binary_error: 0.0831
[1816]	training's binary_error: 5e-05	valid_1's binary_error: 0.0831
[1817]	training's binary_error: 5e-05	valid_1's binary_error: 0.0831
[1818]	training's binary_error: 5e-05	valid_1's binary_error: 0.0831
[1819]	training's binary_error: 5e-05	valid_1's binary_error: 0.0832
[1820]	training's binary_error: 5e-05	valid_1's binary_error: 0.0832
[1821]	training's binary_error: 5e-05	valid_1's binary_error: 0.0833
[1822]	training's binary_error: 5e-05	valid_1's binary_error: 0.0832
[1823]	training's binary_error: 5e-05	valid_1's binary_error: 0.0833
[1824]	training's binary_error: 5e-05	valid_1's binary_error: 0.0833
[1825]	training's binary_error: 5e-05	valid_1's binary_error: 0.0835
[1826]	training's binary_error: 5e-05	valid_1's binary_error: 0.0835
[1827]	training's binary_error: 2.

[1948]	training's binary_error: 0	valid_1's binary_error: 0.0838
[1949]	training's binary_error: 0	valid_1's binary_error: 0.0838
[1950]	training's binary_error: 0	valid_1's binary_error: 0.0838
[1951]	training's binary_error: 0	valid_1's binary_error: 0.0838
[1952]	training's binary_error: 0	valid_1's binary_error: 0.0838
[1953]	training's binary_error: 0	valid_1's binary_error: 0.0837
[1954]	training's binary_error: 0	valid_1's binary_error: 0.0836
[1955]	training's binary_error: 0	valid_1's binary_error: 0.0836
[1956]	training's binary_error: 0	valid_1's binary_error: 0.0836
[1957]	training's binary_error: 0	valid_1's binary_error: 0.0835
[1958]	training's binary_error: 0	valid_1's binary_error: 0.0835
[1959]	training's binary_error: 0	valid_1's binary_error: 0.0836
[1960]	training's binary_error: 0	valid_1's binary_error: 0.0836
[1961]	training's binary_error: 0	valid_1's binary_error: 0.0835
[1962]	training's binary_error: 0	valid_1's binary_error: 0.0834
[1963]	training's binary_

In [51]:
#验证集结果
from sklearn import metrics
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import f1_score 
from sklearn.metrics import confusion_matrix
y_pred0=model.predict(test_final.drop(columns='loan_status'),num_iteration=model.best_iteration)
y_pred=[1 if x >=0.5 else 0 for x in y_pred0]
print('准确率')
print(accuracy_score(test_final['loan_status'],y_pred))
print('f1score')
print(f1_score(test_final['loan_status'],y_pred)) 
print('auc')
print(roc_auc_score(test_final['loan_status'],y_pred))
print('混淆矩阵')
confusion_matrix(test_final['loan_status'],y_pred)

准确率
0.91282
f1score
0.945782907747609
auc
0.8624407787630743
混淆矩阵


array([[ 7621,  2153],
       [ 2206, 38020]])

## 二.随机探索阶段

参考文献  
https://lightgbm.apachecn.org/#/docs/6  
https://zhuanlan.zhihu.com/p/376485485  
https://www.cnblogs.com/bjwu/p/9307344.html  
https://zhuanlan.zhihu.com/p/76206257

In [85]:
#查看sklearn支持的评价指标
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [121]:
#max_depth ：设置树深度，深度越大可能过拟合
#num_leaves：过大可能会导致过拟合
#learning_rate：学习速率
#subsample/bagging_fraction:数据采样
#colsample_bytree/feature_fraction: 特征采样
from sklearn.model_selection import GridSearchCV

model_lgb = lgb.LGBMClassifier(num_leaves=50,boosting_type = 'gbdt',
                              learning_rate=0.1, n_estimators=43, max_depth=6,
                              metric='binary_logloss', bagging_fraction = 0.8,feature_fraction = 0.8)
params_test1={
    'max_depth':range(3,8,2),
    'num_leaves':range(50, 170, 30),
    'bagging_fraction':[0.4,0.6,0.8,1],
    'feature_fraction':[0.4,0.6,0.8,1]
}
score_list=list(['roc_auc','f1_macro','f1_micro','accuracy','precision','recall'])
gsearch_dict={}
import time
#尝试多种评价指标做交叉验证
for score_type in score_list:
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    print(score_type)
    gsearch=GridSearchCV(estimator=model_lgb, param_grid=params_test1, scoring=score_type, cv=5, verbose=0, n_jobs=4)
    gsearch_result=gsearch.fit(train_final.drop(columns='loan_status'),train_final['loan_status'])
    #参数与指标
    print(gsearch_result.best_params_)
    print(gsearch_result.best_score_)
    #混淆矩阵
    y_pred=gsearch_result.predict(train_final.drop(columns='loan_status'))
    print(confusion_matrix(train_final['loan_status'],y_pred))
    gsearch_dict[score_type]=gsearch_result

2021-07-06 19:44:06
roc_auc
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
{'bagging_fraction': 0.4, 'feature_fraction': 0.6, 'max_depth': 5, 'num_leaves': 50}
0.9605377680579815
[[ 8342  1870]
 [ 1974 37814]]
2021-07-06 19:49:47
f1_macro
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
{'bagging_fraction': 0.4, 'feature_fraction': 1, 'max_depth': 5, 'num_leaves': 50}
0.8764860651016686
[[ 8363  1849]
 [ 1992 37796]]
2021-07-06 19:55:24
f1_micro
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.6, c

In [ ]:
#从以上过程中，发现使用roc_auc指标进行寻优，混淆矩阵结果相对可以接受，所以后续使用roc_auc作为寻优指标
#{'bagging_fraction': 0.4, 'feature_fraction': 0.6, 'max_depth': 5, 'num_leaves': 50}作为基本参数，基本符合各个指标下的寻优结果

## 三.顺序搜索阶段

In [142]:
#首先进行学习速率深度和叶子节点的调优
lgb1 = lgb.LGBMClassifier(boosting_type = 'gbdt',n_estimators=43,
                          metric='binary_logloss', bagging_fraction = 0.4,feature_fraction = 0.6)
params_test1={
    'learning_rate':np.linspace(0.02,0.1,5),
    'max_depth':range(1,11,1),
    'num_leaves':range(25,80,5)
}
gsearch1=GridSearchCV(estimator=lgb1, param_grid=params_test1, scoring='roc_auc', cv=5, verbose=0, n_jobs=4)
gsearch_result1=gsearch1.fit(train_final.drop(columns='loan_status'),train_final['loan_status'])
#参数与指标
print(gsearch_result1.best_params_)
print(gsearch_result1.best_score_)
#混淆矩阵
y_pred=gsearch_result1.predict(train_final.drop(columns='loan_status'))
print(confusion_matrix(train_final['loan_status'],y_pred))

[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
{'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 25}
0.9608356076066592
[[ 8381  1831]
 [ 1944 37844]]


In [143]:
#观察参数的趋势
means = gsearch_result1.cv_results_['mean_test_score']
params = gsearch_result1.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 25}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 30}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 35}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 40}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 45}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 50}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 55}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 60}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 65}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 70}
0.942971  with:   {'learning_rate': 0.02, 'max_depth': 1, 'num_leaves': 75}
0.953006  with:   {'learning_rate': 0.02, 'max_depth': 2, 'num_leaves': 25}
0.953006  with:   {'learning_rate': 0.02, 'max_depth': 2, 'num_leaves': 30}
0.953006  wi

In [151]:
#数据显示学习速率小于0.1并不理想 ，尝试提高学习速率
#上一步已经获得的'max_depth': 7, 尝试增加深度
#上一步已经获得的'num_leaves': 25，尝试减少叶片数量
lgb1 = lgb.LGBMClassifier(boosting_type = 'gbdt',n_estimators=43,
                          metric='binary_logloss', bagging_fraction = 0.4,feature_fraction = 0.6)
params_test1={
    'learning_rate':np.linspace(0.1,0.5,5),
    'max_depth':range(7,12,1),
    'num_leaves':range(15,26,1)
}
gsearch1=GridSearchCV(estimator=lgb1, param_grid=params_test1, scoring='roc_auc', cv=5, verbose=0, n_jobs=4)
gsearch_result1=gsearch1.fit(train_final.drop(columns='loan_status'),train_final['loan_status'])
#参数与指标
print(gsearch_result1.best_params_)
print(gsearch_result1.best_score_)
#混淆矩阵
y_pred=gsearch_result1.predict(train_final.drop(columns='loan_status'))
print(confusion_matrix(train_final['loan_status'],y_pred))

[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
{'learning_rate': 0.1, 'max_depth': 11, 'num_leaves': 20}
0.960851962177529
[[ 8360  1852]
 [ 1958 37830]]


In [153]:
#继续观察参数的趋势
means = gsearch_result1.cv_results_['mean_test_score']
params = gsearch_result1.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

0.960752  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 15}
0.960695  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 16}
0.960721  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 17}
0.960744  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 18}
0.960692  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 19}
0.960734  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 20}
0.960806  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 21}
0.960811  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 22}
0.960691  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 23}
0.960754  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 24}
0.960836  with:   {'learning_rate': 0.1, 'max_depth': 7, 'num_leaves': 25}
0.960823  with:   {'learning_rate': 0.1, 'max_depth': 8, 'num_leaves': 15}
0.960765  with:   {'learning_rate': 0.1, 'max_depth': 8, 'num_leaves': 16}
0.960699  with:   {'learn

In [158]:
#然后调整bagging_fraction和feature_fraction
lgb1 = lgb.LGBMClassifier(boosting_type = 'gbdt',n_estimators=43,metric='binary_logloss',
                          learning_rate=0.1, max_depth=11, num_leaves=20)
params_test1={
    'bagging_fraction':[0.4,0.6,0.8,1],
    'feature_fraction':[0.4,0.6,0.8,1]
}
gsearch1=GridSearchCV(estimator=lgb1, param_grid=params_test1, scoring='roc_auc', cv=5, verbose=0, n_jobs=4)
gsearch_result1=gsearch1.fit(train_final.drop(columns='loan_status'),train_final['loan_status'])
#参数与指标
print(gsearch_result1.best_params_)
print(gsearch_result1.best_score_)
#混淆矩阵
y_pred=gsearch_result1.predict(train_final.drop(columns='loan_status'))
print(confusion_matrix(train_final['loan_status'],y_pred))

[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
{'bagging_fraction': 0.4, 'feature_fraction': 0.6}
0.960851962177529
[[ 8360  1852]
 [ 1958 37830]]


In [159]:
#继续观察参数的趋势
means = gsearch_result1.cv_results_['mean_test_score']
params = gsearch_result1.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

0.960243  with:   {'bagging_fraction': 0.4, 'feature_fraction': 0.4}
0.960852  with:   {'bagging_fraction': 0.4, 'feature_fraction': 0.6}
0.960799  with:   {'bagging_fraction': 0.4, 'feature_fraction': 0.8}
0.960674  with:   {'bagging_fraction': 0.4, 'feature_fraction': 1}
0.960243  with:   {'bagging_fraction': 0.6, 'feature_fraction': 0.4}
0.960852  with:   {'bagging_fraction': 0.6, 'feature_fraction': 0.6}
0.960799  with:   {'bagging_fraction': 0.6, 'feature_fraction': 0.8}
0.960674  with:   {'bagging_fraction': 0.6, 'feature_fraction': 1}
0.960243  with:   {'bagging_fraction': 0.8, 'feature_fraction': 0.4}
0.960852  with:   {'bagging_fraction': 0.8, 'feature_fraction': 0.6}
0.960799  with:   {'bagging_fraction': 0.8, 'feature_fraction': 0.8}
0.960674  with:   {'bagging_fraction': 0.8, 'feature_fraction': 1}
0.960243  with:   {'bagging_fraction': 1, 'feature_fraction': 0.4}
0.960852  with:   {'bagging_fraction': 1, 'feature_fraction': 0.6}
0.960799  with:   {'bagging_fraction': 1, 'f

## 顺序搜索最优结果  

params_opt={
    'learning_rate': 0.1, 'max_depth': 11, 'num_leaves': 20,
    'bagging_fraction': 1, 'feature_fraction': 0.4
}

In [160]:
##参数在测试集的结果
print('测试集结果')
y_pred=gsearch_result1.predict(test_final.drop(columns='loan_status'))
print('准确率')
print(accuracy_score(test_final['loan_status'],y_pred))
print('f1score')
print(f1_score(test_final['loan_status'],y_pred)) 
print('auc')
print(roc_auc_score(test_final['loan_status'],y_pred))
print('混淆矩阵')
confusion_matrix(test_final['loan_status'],y_pred)

测试集结果
准确率
0.91654
f1score
0.9479331727949892
auc
0.8732337884364433
混淆矩阵


array([[ 7840,  1934],
       [ 2239, 37987]])

## 四.自动寻优阶段

In [216]:
#首先进行缺失值处理
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data = pd.DataFrame(si.fit_transform(train))
train_data.columns=train.columns
test_data = pd.DataFrame(si.fit_transform(test))
test_data.columns=test.columns
test_data.head()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec,discrete_addr_state_1_one_hot,discrete_addr_state_2_one_hot,discrete_addr_state_3_one_hot,discrete_addr_state_4_one_hot,discrete_addr_state_5_one_hot,discrete_addr_state_6_one_hot,discrete_addr_state_7_one_hot,discrete_addr_state_8_one_hot,discrete_addr_state_9_one_hot,discrete_addr_state_10_one_hot,discrete_addr_state_11_one_hot,discrete_addr_state_12_one_hot,discrete_addr_state_13_one_hot,discrete_addr_state_14_one_hot,discrete_addr_state_15_one_hot,discrete_addr_state_16_one_hot,discrete_addr_state_17_one_hot,discrete_addr_state_18_one_hot,discrete_addr_state_19_one_hot,discrete_addr_state_20_one_hot,discrete_addr_state_21_one_hot,discrete_addr_state_22_one_hot,discrete_addr_state_23_one_hot,discrete_addr_state_24_one_hot,discrete_addr_state_25_one_hot,discrete_addr_state_26_one_hot,discrete_addr_state_27_one_hot,discrete_addr_state_28_one_hot,discrete_addr_state_29_one_hot,discrete_addr_state_30_one_hot,discrete_addr_state_31_one_hot,discrete_addr_state_32_one_hot,discrete_addr_state_33_one_hot,discrete_addr_state_34_one_hot,discrete_addr_state_35_one_hot,discrete_addr_state_36_one_hot,discrete_addr_state_37_one_hot,discrete_addr_state_38_one_hot,discrete_addr_state_39_one_hot,discrete_addr_state_40_one_hot,discrete_addr_state_41_one_hot,discrete_addr_state_42_one_hot,discrete_addr_state_43_one_hot,discrete_addr_state_44_one_hot,discrete_addr_state_45_one_hot,discrete_addr_state_46_one_hot,discrete_addr_state_47_one_hot,discrete_addr_state_48_one_hot,discrete_addr_state_49_one_hot,discrete_application_type_1_one_hot,discrete_application_type_2_one_hot,discrete_emp_length_1_one_hot,discrete_emp_length_2_one_hot,discrete_emp_length_3_one_hot,discrete_emp_length_4_one_hot,discrete_emp_length_5_one_hot,discrete_emp_length_6_one_hot,discrete_emp_length_7_one_hot,discrete_emp_length_8_one_hot,discrete_emp_length_9_one_hot,discrete_emp_length_10_one_hot,discrete_emp_length_11_one_hot,discrete_emp_length_12_one_hot,discrete_grade_1_one_hot,discrete_grade_2_one_hot,discrete_grade_3_one_hot,discrete_grade_4_one_hot,discrete_grade_5_one_hot,discrete_grade_6_one_hot,discrete_grade_7_one_hot,discrete_home_ownership_1_one_hot,discrete_home_ownership_2_one_hot,discrete_home_ownership_3_one_hot,discrete_home_ownership_4_one_hot,discrete_policy_code_1_one_hot,discrete_purpose_1_one_hot,discrete_purpose_2_one_hot,discrete_purpose_3_one_hot,discrete_purpose_4_one_hot,discrete_purpose_5_one_hot,discrete_purpose_6_one_hot,discrete_purpose_7_one_hot,discrete_purpose_8_one_hot,discrete_purpose_9_one_hot,discrete_purpose_10_one_hot,discrete_purpose_11_one_hot,discrete_purpose_12_one_hot,discrete_pymnt_plan_1_one_hot,discrete_sub_grade_1_one_hot,discrete_sub_grade_2_one_hot,discrete_sub_grade_3_one_hot,discrete_sub_grade_4_one_hot,discrete_sub_grade_5_one_hot,discrete_sub_grade_6_one_hot,discrete_sub_grade_7_one_hot,discrete_sub_grade_8_one_hot,discrete_sub_grade_9_one_hot,discrete_sub_grade_10_one_hot,discrete_sub_grade_11_one_hot,discrete_sub_grade_12_one_hot,discrete_sub_grade_13_one_hot,discrete_sub_grade_14_one_hot,discrete_sub_grade_15_one_hot,discrete_sub_grade_16_one_hot,discrete_sub_grade_17_one_hot,discrete_sub_grade_18_one_hot,discrete_sub_grade_19_one_hot,discrete_sub_grade_20_one_hot,discrete_sub_grade_21_one_hot,discrete_sub_grade_22_one_hot,discrete_sub_grade_23_one_hot,discrete_sub_grade_24_one_hot,discrete_sub_grade_25_one_hot,discrete_sub_grade_26_one_hot,discrete_sub_grade_27_one_hot,discrete_sub_grade_28_one_hot,discrete_sub_

In [217]:
import io
import multiprocessing
from contextlib import redirect_stdout
from copy import deepcopy
from dataclasses import dataclass, asdict
import hyperopt.pyll
from hyperopt import fmin, tpe, hp
import numpy as np
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import torch

import copy
cpu_count = 4
use_gpu = False
@dataclass
class LGBOpt:
    num_threads: any = hp.choice('num_threads', [cpu_count])
    num_leaves: any = hp.choice('num_leaves', [64])
    metric: any = hp.choice('metric', ['binary_error'])
    num_round: any = hp.choice('num_rounds', [1000])
    objective: any = hp.choice('objective', ['binary'])
    learning_rate: any = hp.uniform('learning_rate', 0.01, 0.1)
    feature_fraction: any = hp.uniform('feature_fraction', 0.5, 1.0)
    bagging_fraction: any = hp.uniform('bagging_fraction', 0.8, 1.0)
    device_type: any = hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type',
                                                                                   ['cpu'])
    boosting: any = hp.choice('boosting', ['gbdt', 'dart', 'goss'])
    extra_trees: any = hp.choice('extra_tress', [False, True])
    drop_rate: any = hp.uniform('drop_rate', 0, 0.2)
    uniform_drop: any = hp.choice('uniform_drop', [True, False])
    lambda_l1: any = hp.uniform('lambda_l1', 0, 10)  # TODO: Check range
    lambda_l2: any = hp.uniform('lambda_l2', 0, 10)  # TODO: Check range
    min_gain_to_split: any = hp.uniform('min_gain_to_split', 0, 1)  # TODO: Check range
    min_data_in_bin = hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])

    @staticmethod
    def get_common_params():
        return {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 1000, 'learning_rate': 0.01, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
    

In [218]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")


In [219]:

class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt: LGBOpt = None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = LGBOpt()
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def search(self, train_df, eval_df, use_best_eval=True):
        self.opt_params = dict()

        def train_impl(params):
            self.train(train_df, eval_df, params, use_best_eval)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            return self.get_loss(eval_df[self.label], y_pred)

        self.opt_params = fmin(train_impl, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, asdict(self.opt), algo=tpe.suggest, max_evals=self.max_eval)

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models

In [237]:
#训练
fitter = LGBFitter(label='loan_status')
params = {'num_thread': 4, 'num_leaves': 12, 'metric': 'binary', 'objective': 'binary',
                'num_round': 2000, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8}
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)
fitter_result=fitter.train_k_fold(kfold, train_data, test_data, params = params)

The minimum is attained in round 490
Finished loading model, total used 2000 iterations
The minimum is attained in round 341
Finished loading model, total used 2000 iterations
The minimum is attained in round 340
Finished loading model, total used 2000 iterations
The minimum is attained in round 329
Finished loading model, total used 2000 iterations
The minimum is attained in round 399
Finished loading model, total used 2000 iterations


In [274]:
fitter_result

(array([0.47663437, 0.99706559, 0.86079002, ..., 0.9959425 , 0.97990469,
        0.9976346 ]),
 array([0.25114901, 0.7358211 , 0.04692949, ..., 1.186431  , 1.16534993,
        1.18808243]),
 [0.08350000000000002,
  0.07962499999999995,
  0.08399999999999996,
  0.08399999999999996,
  0.07825000000000004],
  <lightgbm.basic.Booster at 0x7fe1055b88d0>])

In [301]:
#查看验证集在各个选出的模型的效果
p=0
y_test=test_data['loan_status']
for gbm1 in fitter_result[3]:
    print(p)
    y_pred0 = gbm1.predict(test_data.drop(columns='loan_status'))
    y_pred=[1 if x >=0.5 else 0 for x in y_pred0]
    y_train=train_data['loan_status']
    print('准确率')
    print(accuracy_score(y_test,y_pred))
    print('f1score')
    print(f1_score(y_test,y_pred)) 
    print('auc')
    print(roc_auc_score(y_test,y_pred))
    print('混淆矩阵')
    print(confusion_matrix(y_test,y_pred))
    p=p+1

0
准确率
0.9158
f1score
0.9458032955715757
auc
0.9050487635994948
混淆矩阵
[[1811  231]
 [ 611 7347]]
1
准确率
0.9191
f1score
0.9489493279485076
auc
0.8818202394106769
混淆矩阵
[[1672  370]
 [ 439 7519]]
2
准确率
0.9161
f1score
0.9462282894315195
auc
0.8994123531498252
混淆矩阵
[[1779  263]
 [ 576 7382]]
3
准确率
0.9209
f1score
0.9498700804867228
auc
0.8907783862338984
混淆矩阵
[[1715  327]
 [ 464 7494]]
4
准确率
0.9074
f1score
0.9397841071660815
auc
0.906506096280694
混淆矩阵
[[1848  194]
 [ 732 7226]]


## 自动调参最优结果

In [320]:
#从以上结果看 ，首个模型效果较好，所以使用首个模型参数 
lgm_best=fitter_result[3][0]
#对应测试集效果
y_pred0=lgm_best.predict(test_final.drop(columns='loan_status'))
y_pred=[1 if x >=0.5 else 0 for x in y_pred0]
print('准确率')
print(accuracy_score(test_final['loan_status'],y_pred))
print('f1score')
print(f1_score(test_final['loan_status'],y_pred)) 
print('auc')
print(roc_auc_score(test_final['loan_status'],y_pred))
print('混淆矩阵')
confusion_matrix(test_final['loan_status'],y_pred)

准确率
0.9122
f1score
0.9442468884937771
auc
0.8935787102034545
混淆矩阵


array([[ 8435,  1339],
       [ 3051, 37175]])

In [349]:
#通过文件查看参数
lgm_best.save_model('model.txt')

In [353]:
"""
parameters:
[boosting: gbdt]
[objective: binary]
[metric: binary_logloss]
[tree_learner: serial]
[device_type: cpu]
[data: ]
[valid: ]
[num_iterations: 2000]
[learning_rate: 0.02]
[num_leaves: 12]
[num_threads: 4]
[deterministic: 0]
[force_col_wise: 0]
[force_row_wise: 0]
[histogram_pool_size: -1]
[max_depth: -1]
[min_data_in_leaf: 20]
[min_sum_hessian_in_leaf: 0.001]
[bagging_fraction: 0.8]
[pos_bagging_fraction: 1]
[neg_bagging_fraction: 1]
[bagging_freq: 0]
[bagging_seed: 3]
[feature_fraction: 0.8]
[feature_fraction_bynode: 1]
[feature_fraction_seed: 2]
[extra_trees: 0]
[extra_seed: 6]
[early_stopping_round: 0]
[first_metric_only: 0]
[max_delta_step: 0]
[lambda_l1: 0]
[lambda_l2: 0]
[min_gain_to_split: 0]
[drop_rate: 0.1]
[max_drop: 50]
[skip_drop: 0.5]
[xgboost_dart_mode: 0]
[uniform_drop: 0]
[drop_seed: 4]
[top_rate: 0.2]
[other_rate: 0.1]
[min_data_per_group: 100]
[max_cat_threshold: 32]
[cat_l2: 10]
[cat_smooth: 10]
[max_cat_to_onehot: 4]
[top_k: 20]
[monotone_constraints: ]
[monotone_constraints_method: basic]
[monotone_penalty: 0]
[feature_contri: ]
[forcedsplits_filename: ]
[refit_decay_rate: 0.9]
[cegb_tradeoff: 1]
[cegb_penalty_split: 0]
[cegb_penalty_feature_lazy: ]
[cegb_penalty_feature_coupled: ]
[path_smooth: 0]
[interaction_constraints: ]
[verbosity: 1]
[saved_feature_importance_type: 0]
[max_bin: 255]
[max_bin_by_feature: ]
[min_data_in_bin: 3]
[bin_construct_sample_cnt: 200000]
[data_random_seed: 1]
[is_enable_sparse: 1]
[enable_bundle: 1]
[use_missing: 1]
[zero_as_missing: 0]
[feature_pre_filter: 1]
[pre_partition: 0]
[two_round: 0]
[header: 0]
[label_column: ]
[weight_column: ]
[group_column: ]
[ignore_column: ]
[categorical_feature: ]
[forcedbins_filename: ]
[objective_seed: 5]
[num_class: 1]
[is_unbalance: 0]
[scale_pos_weight: 1]
[sigmoid: 1]
[boost_from_average: 1]
[reg_sqrt: 0]
[alpha: 0.9]
[fair_c: 1]
[poisson_max_delta_step: 0.7]
[tweedie_variance_power: 1.5]
[lambdarank_truncation_level: 30]
[lambdarank_norm: 1]
[label_gain: ]
[eval_at: ]
[multi_error_top_k: 1]
[auc_mu_weights: ]
[num_machines: 1]
[local_listen_port: 12400]
[time_out: 120]
[machine_list_filename: ]
[machines: ]
[gpu_platform_id: -1]
[gpu_device_id: -1]
[gpu_use_dp: 0]
[num_gpu: 1]
"""

'\nparameters:\n[boosting: gbdt]\n[objective: binary]\n[metric: binary_logloss]\n[tree_learner: serial]\n[device_type: cpu]\n[data: ]\n[valid: ]\n[num_iterations: 2000]\n[learning_rate: 0.02]\n[num_leaves: 12]\n[num_threads: 4]\n[deterministic: 0]\n[force_col_wise: 0]\n[force_row_wise: 0]\n[histogram_pool_size: -1]\n[max_depth: -1]\n[min_data_in_leaf: 20]\n[min_sum_hessian_in_leaf: 0.001]\n[bagging_fraction: 0.8]\n[pos_bagging_fraction: 1]\n[neg_bagging_fraction: 1]\n[bagging_freq: 0]\n[bagging_seed: 3]\n[feature_fraction: 0.8]\n[feature_fraction_bynode: 1]\n[feature_fraction_seed: 2]\n[extra_trees: 0]\n[extra_seed: 6]\n[early_stopping_round: 0]\n[first_metric_only: 0]\n[max_delta_step: 0]\n[lambda_l1: 0]\n[lambda_l2: 0]\n[min_gain_to_split: 0]\n[drop_rate: 0.1]\n[max_drop: 50]\n[skip_drop: 0.5]\n[xgboost_dart_mode: 0]\n[uniform_drop: 0]\n[drop_seed: 4]\n[top_rate: 0.2]\n[other_rate: 0.1]\n[min_data_per_group: 100]\n[max_cat_threshold: 32]\n[cat_l2: 10]\n[cat_smooth: 10]\n[max_cat_to_o